<a href="https://colab.research.google.com/github/cheng1103/Test/blob/main/Cnyes_category_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><span style="font-size:20px"><strong>鉅亨網新聞主題爬蟲（精確版，用於Colab平台）</strong></span></h1>

<p>可自訂主題新聞與日期區間爬取新聞內文，並可設定是否爬取頭條以外的新聞<br />
<span style="color:null"><strong>輸出資料的欄位依序為：張貼時間(時間戳)、張貼時間、版別、標題、撰文者、內文、標籤、網址、頭條標記</strong></span><strong>。資料格式為csv檔，並以UTF8編碼，直接存入Google雲端硬碟</strong></p>

<p>支援的分類新聞目錄：</p>

<p>1.台股 V<br />
&nbsp; a.台灣政經 V<br />
&nbsp; b.台股盤勢 V<br />
&nbsp; c.台股盤勢 V<br />
&nbsp; d.台股盤勢 V</p>

<p>2.國際股 V<br />
&nbsp; a.美股 V<br />
&nbsp; b.美股雷達 V<br />
&nbsp; c.歐亞股 V<br />
&nbsp; d.國際政經 V</p>

<p>3.陸港股 V</p>

<p>4.外匯 V</p>

<p>5.期貨 V</p>

<p>6.房產 V</p>

<p>7.基金 V</p>

<p>8.區塊鏈 V</p>


# 新聞類別與查詢區間設定


In [ ]:
#設定區域
#==============================================
#頭條設定 
# 1表示限定頭條； 0表示不限 
Headline = 1

#輸入類別
c = '台股'

#查詢區間
s_time = "2022-01-01 00:00:00" #開始日期 
e_time = "2022-01-03 00:00:00" #結束日期

#檔名設定
file_name = 'original_data'

#==============================================

data_URL = []

kind = {
  '台股':'tw_stock',
  '台灣政經':'tw_macro',
  '台股盤勢':'tw_quo',
  '專家觀點':'stock_report',
  '台股新聞':'tw_stock_news',

  '國際股':'wd_stock',
  '美股':'us_stock',
  '美股雷達':'us_stock_live',
  '歐亞股':'eu_asia_stock',
  '國際政經':'wd_macro',

  '陸港股':'cn_stock',
  '外匯':'forex',
  '期貨':'future',
  '房產':'cnyeshouse',
  '基金':'fund',
  '區塊鏈':'bc'

}

c = kind.get(c)


#頭條篩選
if Headline == 1 :
  CH = '&isCategoryHeadline=1'
else :
  CH = ''

# 安裝並載入套件

In [2]:
!pip install fake_useragent
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
from lxml import etree

import random
import time
import datetime
import json
import pandas as pd
#%reset -f

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=aa1c002c0b8a0ae60893693f45c484a598d870ec58847e8aa86b24baa36b49e2
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


# 自定義程式：爬取鉅亨網新聞網址清單

In [ ]:
def url_list(s_time,e_time):
  
  #==============================================
  #時間區間設定
  s_time = int(time.mktime(time.strptime(s_time, "%Y-%m-%d %H:%M:%S")))-28800 # 轉成時間戳
  e_time = int(time.mktime(time.strptime(e_time, "%Y-%m-%d %H:%M:%S")))-28800

  #==============================================
  p = 1
  l_page = 9
  data = pd.DataFrame()

  while p <= l_page :

    headers = UserAgent().random #隨機請求表頭
    
    url = 'https://api.cnyes.com/media/api/v1/newslist/category/'+c+'?startAt='+str(s_time)+'&endAt='+str(e_time)+CH+"&limit=30&page="+str(p)
    req = requests.get(url, headers={"User-Agent":headers})
    req = req.json()

    #最末頁之路徑
    if p == 1:
      l_page = req["items"]["last_page"] 

    req_data = req["items"]["data"]
    temp_data = pd.DataFrame.from_dict(req_data)
    
    #擷取資料
    publishAt = temp_data['publishAt']
    newsId = temp_data['newsId']
    title = temp_data['title']
    #isIndex = temp_data['isIndex'] 
    isCategoryHeadline = temp_data['isCategoryHeadline']
    #categoryId = temp_data['categoryId']
    
    temp_data = pd.concat([publishAt, newsId, title, isCategoryHeadline], axis=1)
    data = pd.concat([data, temp_data], ignore_index=True)
    
    

    #休眠
    t = random.uniform(0.005, 0.1) #隨機數
    time.sleep(t) #暫停

    p = p+1

    if p%20 == 0:
      print(p/l_page*100)
      data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
      time.sleep(0.5)

  data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
  return(data)



# 執行程式

In [ ]:
data_URL = url_list(s_time,e_time)
data_URL = data_URL.sort_values(by='publishAt', ascending=True)
data_URL = data_URL.reset_index(drop=True) 

# 檢視新聞網址資料

In [ ]:
data_URL

,publishAt,newsId,title,isCategoryHeadline
0,1640970025,4794194,〈元旦新制〉洗發票不給獎、巧克力正名 15項民生新規定一次看,1
1,1640991635,4794474,〈元旦新制〉保單借款紓困、理專十誡2.0 八大金融新規上路,1
2,1641019388,4795332,世界先進向友達購入L3B廠 完成交割,1
3,1641021326,4795333,〈圖片集〉台北101跨年倒數有亮點！中國信託銀行動畫字樣迎接2022年,1
4,1641024044,4795145,〈觀察〉中國鋼鐵業加速整併 台廠喜憂參半,1
...,...,...,...,...
84,1641219047,4796126,今年電價調漲壓力大 經長估國際油價、天然氣Q1回跌,1
85,1641219165,4796125,同泰前執行長李遠智等人涉違法 財務損失2000萬元,1
86,1641219196,4796127,鴻海官方臉書正式開張 電動巴士Model T打頭陣,1
87,1641219226,4796143,〈0103強震〉全台有感 面板雙虎營運未受影響,1


# 爬取新聞內文

In [ ]:
data = pd.DataFrame()
l_page = len(data_URL)

for i in range(l_page):

  headers = UserAgent().random
  url = data_URL['newsId'][i]
  url = 'https://news.cnyes.com/news/id/'+str(url)

  req = requests.get(url, headers={"User-Agent":headers})
  req = req.content.decode()
  req = etree.HTML(req)


  catalog = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/div[1]/nav/a/span/span/text()')
  catalog = ",".join(catalog)

  title = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/h1/text()')
  title = "".join(title)

  reporter = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/div[2]/span/span/text()')
  reporter = "".join(reporter)


  d_time = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/div[2]/time/text()')
  d_time = "".join(d_time)

  content = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/div[2]/div/p/text() | //*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/div[2]/div/p/a/text()')
  content = "".join(content)
  if len(content) == 0:
    content = '0'

  tag = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/nav/a/span/text()')
  tag = ",".join(tag)

  publishAt = data_URL['publishAt'][i]
  isCategoryHeadline = data_URL['isCategoryHeadline'][i]


  temp_data = {(publishAt ,d_time, catalog, title, reporter, content, tag, isCategoryHeadline, url)}
  temp_data = pd.DataFrame(temp_data)
  data = data.append(temp_data)

  t = random.uniform(0.005, 0.1) #隨機數
  time.sleep(t) #暫停

  #列印出進度
  if i % 50 == 0:
    print(i/l_page*100)
    t = random.uniform(0.1, 0.5) #隨機數
    time.sleep(t)




0.0
56.17977528089888


In [6]:
  headers = UserAgent().random
  url = 'https://news.cnyes.com/news/id/4795333'

  req = requests.get(url, headers={"User-Agent":headers})
  req = req.content.decode()
  req = etree.HTML(req)
  content = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/div[2]/div/p/text() | //*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/div[2]/div/p/a/text()')
  content = "".join(content)
  content

''

# 檢視新聞內文資料

In [ ]:
data.columns = [ 'publishAt', 'time', 'category', 'title', 'reporter', 'content', 'tag', 'Headline', 'url']
data
#[publishAt, Headline, categoryName, title, content, stock, newsId_url], axis=1)

,publishAt,time,category,title,reporter,content,tag,Headline,url
0,1640970025,2022/01/01 01:00,鉅亨新視界,〈元旦新制〉洗發票不給獎、巧克力正名 15項民生新規定一次看,鉅亨網新聞中心,邁入 2022 年，國內又有許多新制上路，其中勞權方面影響最多，包括請領勞保年金從 62 歲...,"元旦,巧克力,新制,疫苗,基本工資,自販機,勞保,老年年金,洗選蛋,國道,勞工,值班,水稻收入保險",1,https://news.cnyes.com/news/id/4794194
0,1640991635,2022/01/01 07:00,鉅亨新視界,〈元旦新制〉保單借款紓困、理專十誡2.0 八大金融新規上路,鉅亨網新聞中心,2022 年 1 月起，金管會、財政部又有多項金融新制上路，其中，理專十誡 2.0 版本將強...,"金管會,保單借款,紓困,元旦,壽險業,遺產稅,當沖,投信",1,https://news.cnyes.com/news/id/4794474
0,1641019388,2022/01/01 14:43,"台股,台股新聞",世界先進向友達購入L3B廠 完成交割,鉅亨網記者林薏茹 台北,晶圓代工廠世界先進 (5347-TW) 今 (1) 日宣佈，向友達 (2409-TW) 購入...,"世界先進,半導體,晶圓代工,友達",1,https://news.cnyes.com/news/id/4795332
0,1641021326,2022/01/01 15:15,"台股,台股新聞",〈圖片集〉台北101跨年倒數有亮點！中國信託銀行動畫字樣迎接2022年,鉅亨網記者陳蕙綾 台北,,"台北101,跨年,中國信託",1,https://news.cnyes.com/news/id/4795333
0,1641024044,2022/01/01 16:00,鉅亨新視界,〈觀察〉中國鋼鐵業加速整併 台廠喜憂參半,鉅亨網記者彭昱文 台北,中國積極整頓當地鋼鐵產業，不僅去化產能降排外，十四五計畫更推動鋼鐵產業進行整併、打造超大型鋼...,"中國,十四五,鋼鐵",1,https://news.cnyes.com/news/id/4795145
...,...,...,...,...,...,...,...,...,...
0,1641219047,2022/01/03 22:10,"台股,台灣政經",今年電價調漲壓力大 經長估國際油價、天然氣Q1回跌,鉅亨網新聞中心,立法院預算中心指出，國內綠能在 2025 年發電占比無法達標 2 成，僅會達到 15.2%，...,"天然氣,油價,能源,王美花,經濟部",1,https://news.cnyes.com/news/id/4796126
0,1641219165,2022/01/03 22:12,"台股,台股新聞",同泰前執行長李遠智等人涉違法 財務損失2000萬元,鉅亨網記者張欽發 台北,軟板廠同泰 (3321-TW) 今 (3) 日發布重大訊息指出，前執行長李遠智及員工李家銘違...,"欣興,同泰,證交法",1,https://news.cnyes.com/news/id/4796125
0,1641219196,2022/01/03 22:13,"台股,台股新聞",鴻海官方臉書正式開張 電動巴士Model T打頭陣,鉅亨網記者彭昱文 台北,鴻海 (2317-TW) 持續強化對外溝通，官方 Facebook 粉絲團也在今 (3) 日...,"鴻海,電動巴士,臉書,Model T",1,https://news.cnyes.com/news/id/4796127
0,1641219226,2022/01/03 22:13,"台股,台股新聞",〈0103強震〉全台有感 面板雙虎營運未受影響,鉅亨網記者劉韋廷 台北,今 (3) 日晚間台灣東部海域發生芮氏規模 6.0 有感地震，全台有感，雙北、宜蘭縣最大震度...,"地震,面板,友達,群創",1,https://news.cnyes.com/news/id/4796143


# 連結Google Drive並匯出資料

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


#輸出檔案
path = '/content/drive/MyDrive/Colab Notebooks/'+file_name+'_full.csv'
data.to_csv(path)

Mounted at /content/drive
